In [1]:
import torch
import matplotlib.pyplot as plt
import numpy as np
import os
import json
from tqdm import tqdm

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [2]:
from bcnf.models.cnf import CondRealNVP_v2
from bcnf.utils import get_dir
from bcnf.utils import load_config, load_data, sub_root_path, inn_nll_loss
from bcnf.eval.calibration import compute_y_hat_ranks, compute_CDF_residuals

from bcnf.train.trainer_data_handler import TrainerDataHandler
from bcnf.train.utils import get_data_type

## Model

In [3]:
MODEL_NAME = f'trajectory_TRF_small'

In [4]:
with open(os.path.join(get_dir('models', 'bcnf-models', MODEL_NAME), 'config.json'), 'r') as f:
    config = load_config(json.load(f)['config_path'])
    config['data']['path'] = sub_root_path(config['data']['path']).replace('/train', '/test')
    print(config['data']['path'])

cnf = CondRealNVP_v2.from_config(config).to(device)

cnf.load_state_dict(torch.load(os.path.join(get_dir('models', 'bcnf-models', MODEL_NAME), "state_dict.pt")))

cnf.eval();

/home/psaegert/Projects/bcnf/data/bcnf-data/fixed_data_render_2s_15FPS/test


In [5]:
print(f'Number of parameters: {cnf.n_params:,}')

Number of parameters: 128,129


In [6]:
parameter_selection = cnf.parameter_index_mapping.parameters

## Data

In [7]:
T = 2
dt = 1/15

In [8]:
data = load_data(sub_root_path(config['data']['path']).replace('/train', '/test'), verbose=True, keep_output_type='trajectories')
# data = load_data(sub_root_path(config['data']['path']).replace('/train', '/test'), verbose=True, keep_output_type='videos')

Loading data from directory: 100%|██████████| 1/1 [00:18<00:00, 18.58s/it, file=fixed_data_render_2s_15FPS_5.pkl]


In [9]:
y, conditions = TrainerDataHandler().get_data_for_training(
    config,
    cnf.parameter_index_mapping,
    dtype=get_data_type(dtype=config["global"]["dtype"]),
    return_tensor_dataset=False)

print(f'{y.shape=}')
for condition in conditions:
    print(f'{condition.shape=}')

y.shape=torch.Size([1000, 19])
condition.shape=torch.Size([1000, 30, 3])


In [20]:
y_0_dict = cnf.parameter_index_mapping.dictify(y[1])
y_0_dict

{'x0_x': tensor(-11.2364, device='cuda:0'),
 'x0_y': tensor(9.0433, device='cuda:0'),
 'x0_z': tensor(2.2730, device='cuda:0'),
 'v0_x': tensor(13.6815, device='cuda:0'),
 'v0_y': tensor(-3.3400, device='cuda:0'),
 'v0_z': tensor(9.6067, device='cuda:0'),
 'g': tensor(-9.6333, device='cuda:0'),
 'w_x': tensor(2.5367, device='cuda:0'),
 'w_y': tensor(-3.7914, device='cuda:0'),
 'w_z': tensor(-1.1655, device='cuda:0'),
 'b': tensor(0.0003, device='cuda:0'),
 'm': tensor(0.1260, device='cuda:0'),
 'a_x': tensor(0., device='cuda:0'),
 'a_y': tensor(-0., device='cuda:0'),
 'a_z': tensor(0., device='cuda:0'),
 'r': tensor(0.0316, device='cuda:0'),
 'A': tensor(0.0031, device='cuda:0'),
 'Cd': tensor(0.0584, device='cuda:0'),
 'rho': tensor(1.4308, device='cuda:0')}

In [21]:
def wind_acceleration(rho, Cd, r, m, w_x, w_y, w_z):
    A = np.pi * r**2
    Fd = 0.5 * rho * Cd * A * (w_x**2 + w_y**2 + w_z**2)
    return (Fd) / m

In [22]:
wind_acceleration(y_0_dict['rho'], y_0_dict['Cd'], y_0_dict['r'], y_0_dict['m'], y_0_dict['w_x'], y_0_dict['w_y'], y_0_dict['w_z'])

tensor(0.0231, device='cuda:0')

In [28]:
wind_acceleration(y_0_dict['rho'], y_0_dict['Cd'], y_0_dict['r'], y_0_dict['m'], y_0_dict['v0_x'], y_0_dict['v0_y'], y_0_dict['v0_z'])

tensor(0.3033, device='cuda:0')

In [27]:
# Determine the magnitude of the initial velocity
v_0 = torch.sqrt(y_0_dict['v0_x']**2 + y_0_dict['v0_y']**2 + y_0_dict['v0_z']**2)

print(f'v_0 [km/h]: {v_0 * 3.6:.2f}')

v_0 [km/h]: 61.37


In [29]:
# Determine the magnitude of the wind
v_0 = torch.sqrt(y_0_dict['w_x']**2 + y_0_dict['w_y']**2 + y_0_dict['w_z']**2)

print(f'v_0 [km/h]: {v_0 * 3.6:.2f}')

v_0 [km/h]: 16.95
